In [1]:
from collections import Counter
from collections import defaultdict

import numpy as np
from pandas.api.types import CategoricalDtype
import pandas as pd
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_colwidth', 2000)

from tqdm import tqdm

import implicit
from scipy.sparse import csr_matrix

In [2]:
megarelation = pd.read_csv('../data/raw/кружки/MegaRelation_hackaton.csv', sep=';')
classificator = pd.read_csv('../data/raw/кружки/Classificator_hachaton.csv', sep=';')
org = pd.read_csv('../data/raw/кружки/org_hackaton.csv', sep=';')
services = pd.read_csv('../data/raw/кружки/services_hackaton.csv', sep=';')

In [3]:
l = []
for idx, row in classificator.iterrows():
    id_ = row['id_классификатора']
    if pd.isna(id_):
        l.append('')
    else:
        x = classificator[classificator['id_классификатора'] == int(id_)]['Наименование'].tolist()[0]
        l.append(x)
        
classificator['наименование_классификатора'] = l

In [4]:
l = []
for idx, row in classificator.iterrows():
    parent_id = row['id_родительского_классификатора']
    if pd.isna(parent_id):
        l.append('')
    else:
        x = classificator[classificator['id_классификатора'] == int(parent_id)]['Наименование'].tolist()[0]
        l.append(x)
        
classificator['наименование_родительского_классификатора'] = l

In [5]:
megarelation = megarelation[[
    'id_зачисления',
    'id_ученика',
    'id_организации',
    'id_заявления',
    'id_услуги',
]]

# classificator = classificator[[
#     'id_ученика',
#     'id_организации',
#     'id_услуги',
# ]]

services = services[[
    'id_услуги',
    'Классификатор_услуги',
    'id_организации',
    'Наименование_услуги',
]]

megarelation = megarelation.drop_duplicates()
#classificator = classificator.drop_duplicates()
org = org.drop_duplicates()
services = services.drop_duplicates()

In [6]:
# Оставлю только один, а то по два метро на один орг указано часто:
print(org.shape)
org = org[org.index.isin(org.drop('Метро', axis=1).drop_duplicates().index)]
print(org.shape)

(442, 5)
(302, 5)


In [7]:
df = pd.merge(megarelation, services, on='id_услуги')
df['id_организации'] = df['id_организации_x']
df = df.drop(columns=['id_организации_x', 'id_организации_y'])
df = pd.merge(df, org, on='id_организации')

In [8]:
df.head(2)

,id_зачисления,id_ученика,id_заявления,id_услуги,Классификатор_услуги,Наименование_услуги,id_организации,полное_наименование,краткое_наименование,улица,Метро
0,1325954,25969.0,950832.0,39178,3000001,Архитектура,30,"Государственное бюджетное учреждение дополнительного образования города Москвы ""Детская школа искусств ""СТАРТ"" архитектурно-художественного профиля""","ГБУДО г. Москвы ""ДШИ ""СТАРТ""",Зоологическая улица,Баррикадная
1,2879918,158129.0,989658.0,39178,3000001,Архитектура,30,"Государственное бюджетное учреждение дополнительного образования города Москвы ""Детская школа искусств ""СТАРТ"" архитектурно-художественного профиля""","ГБУДО г. Москвы ""ДШИ ""СТАРТ""",Зоологическая улица,Баррикадная


In [9]:
classificator['Классификатор_услуги'] = classificator['id_классификатора']
classificator = classificator.drop(columns=['id_классификатора'])

In [10]:
df = pd.merge(df, classificator, on='Классификатор_услуги')

Помёрджил!

In [11]:
print(df.shape)
df = df.dropna(subset=['id_ученика', 'id_услуги', 'Метро', 'id_заявления'])
print(df.shape)

(645286, 15)
(514163, 15)


In [12]:
df['id_ученика'] = df['id_ученика'].astype(int)
df['id_заявления'] = df['id_заявления'].astype(int)
df['id_услуги'] = df['id_услуги'].astype(int)

df['id_родительского_классификатора'] = df['id_родительского_классификатора'].fillna(0)
df['id_родительского_классификатора'] = df['id_родительского_классификатора'].astype(int)

df['id_родительского_классификатора'] = df['id_родительского_классификатора'].fillna(0)
df['id_родительского_классификатора'] = df['id_родительского_классификатора'].astype(int)

In [13]:
df = df.drop(columns=['Наименование'])

In [14]:
df.rename({
    'id_зачисления': 'zachislenie_id',
    'id_ученика': 'user_id',
    'id_заявления': 'zayavlenie_id',
    'id_услуги': 'service_id',
    'Классификатор_услуги': 'service_classificator_id',
    'Наименование_услуги': 'service_name',
    'id_организации': 'org_id',
    'полное_наименование': 'org_name_long',
    'краткое_наименование': 'org_name_short',
    'улица': 'org_street',
    'Метро': 'org_metro',
    'id_родительского_классификатора': 'service_classificator_parent_id',
    'наименование_классификатора': 'service_classificator_name',
    'наименование_родительского_классификатора': 'service_classificator_parent_name',
}, axis = 'columns',inplace=True)

In [15]:
df = df[[
    'zachislenie_id',
    'zayavlenie_id',
    'user_id',
    'service_id',
    'service_name',
    'service_classificator_id',
    'service_classificator_name',
    'service_classificator_parent_id',
    'service_classificator_parent_name',
    'org_id',
    'org_name_short',
    'org_name_long',
    'org_street',
    'org_metro',
]]

## метро

In [16]:
#https://data.mos.ru/classifier/7704786030-stantsii-moskovskogo-metropolitena

In [17]:
metro = pd.read_csv('../data/interim/data-6467-2020-10-26.csv', encoding='cp1251', sep=';')

Для каждой станции получу ближайшие к ней, то есть из её же района:

In [18]:
metro[metro['Station'] == 'Авиамоторная']['District'].tolist()

['район Лефортово', 'район Лефортово', 'район Лефортово']

In [19]:
metro2nearest = {k: [] for k in sorted(metro['Station'].unique())}

#for st in metro[metro['District'] == metro[metro['Line'] == k]['District'][0]]['Line'].unique()

for k in metro2nearest:
    district = metro[metro['Station'] == k]['District'].tolist()[0]
    stations = sorted([x for x in metro[metro['District'] == district]['Station'].unique().tolist()])
    metro2nearest[k] = stations

In [21]:
df['nearest_metro'] = df['org_metro'].map(metro2nearest)

## text

In [22]:
df['text'] = df['service_classificator_parent_name'] + '__'  + df['service_classificator_name'] + '__' + df['service_name']

In [23]:
df.to_parquet('../data/interim/sections.parquet')

## BayesianPersonalizedRanking

In [1]:
from collections import Counter
from collections import defaultdict
import pickle

import numpy as np
import pandas as pd
from pandas.api.types import CategoricalDtype

import implicit
from scipy.sparse import csr_matrix

In [7]:
PATH_TO_SECTIONS_DF = '../data/processed/sections.parquet'
PATH_TO_SECTIONS_MODEL = '../data/processed/sections_model.pickle'

In [3]:
def prepare_df(df):
    data = df.copy()
    data = data.dropna(subset=['user_id', 'service_id'])
    data = data.drop_duplicates(subset=['user_id', 'service_id'])
    data = data[['user_id', 'service_id']]
    return data

In [4]:
df = pd.read_parquet(PATH_TO_SECTIONS_DF)
data = prepare_df(df)

In [5]:
c_item = CategoricalDtype(sorted(data['service_id'].unique()), ordered=True)
c_user = CategoricalDtype(sorted(data['user_id'].unique()), ordered=True)

data['service_id'] = data['service_id'].astype(c_item)
data['user_id'] = data['user_id'].astype(c_user)

In [6]:
# to categorical
data['user_id'] = data['user_id'].astype(c_user)
data['service_id'] = data['service_id'].astype(c_item)

train_data = csr_matrix(
    (np.ones(len(data)), (data['user_id'].cat.codes, data['service_id'].cat.codes)),
    shape=(len(c_user.categories), len(c_item.categories))
)

CONFIDENCE = 2
model = implicit.bpr.BayesianPersonalizedRanking(
    factors=200,
    iterations=500,
    random_state=567,
    verify_negative_samples=True,
)

model.fit(CONFIDENCE * train_data.transpose())

In [12]:
with open(PATH_TO_SECTIONS_MODEL, 'wb') as f:
    pickle.dump(model, f)

In [14]:
with open(PATH_TO_SECTIONS_MODEL, 'rb') as f:
    model = pickle.load(f)

In [29]:
random_state = None
name = data['user_id'].sample(n=1, random_state=random_state).tolist()[0]
user_id = list(c_user.categories).index(name)

recommended_items = model.recommend(user_id, train_data, N=10,
                                    filter_already_liked_items=True,
                                    #recalculate_user=True
                                   )
recommended_items = [list(c_item.categories)[x[0]] for x in recommended_items]
already = data[data['user_id'] == name]['service_id'].unique().tolist()

In [30]:
# print('ALREADY')
# print(already), 

# print()

# print('RECOMMENDED')
# print(recommended_items)

print('ALREADY')
print([df[df['service_id'] == x]['service_name'].unique().tolist()[0] for x in already])

print()

print('RECOMMENDED')
print([df[df['service_id'] == x]['service_name'].unique().tolist()[0] for x in recommended_items])

# на вход список айдишник  того что у юзера было
# на выходе список рекомендаций (название, описание, урл картинки)
# для каждого большого класса можно картинку крутую придумать

ALREADY
['Студия художественного творчества', 'Студия художественного творчества', 'Студия художественного творчества.', 'Студия художественного творчества.', 'Кружок  "Английский язык для школьников"', 'Английский язык для школьников.']

RECOMMENDED
['Студия танцевального фитнеса "Зумба"', 'Студия Оригами', 'Студия художественного творчества.', ' Студия современного танца "Мастер Данс"', 'Оригами', 'Секция шахмат.', 'Таэквон-до секция', 'Песочная анимация.', 'Секция "Шахматы".', 'Колибри.']


In [31]:
from pymystem3 import Mystem

mystem = Mystem()

def clean_text(text):
    try:
        tokens = mystem.lemmatize(text.lower())
    except BrokenPipeError:
        mystem.close
        mystem.start
        tokens = mystem.lemmatize(text.lower())
    
    return [x for x in tokens if x not in ['\n', ' ']]


def remove_dublicates_by_tokens(already, recomendations):
    cleaned_already = [clean_text(x) for x in already]
    cleaned_recomendations = [clean_text(x) for x in recomendations]
    idxs = [idx for idx, x in enumerate(cleaned_recomendations) if x not in cleaned_already]
    return [recomendations[idx] for idx in idxs]

In [33]:
recommended = [df[df['service_id'] == x]['service_name'].unique().tolist()[0] for x in recommended_items]
already = [df[df['service_id'] == x]['service_name'].unique().tolist()[0] for x in already]
recommended = remove_dublicates_by_tokens(already, recommended)

Сделаю чтобы по айдишнику отдавала текст.

## Код для предикта

In [1]:
import pickle

import numpy as np
import pandas as pd
from pandas.api.types import CategoricalDtype

from scipy.sparse import csr_matrix

from pymystem3 import Mystem


PATH_TO_SECTIONS_DF = 'sections.parquet'
PATH_TO_SECTIONS_MODEL = 'sections_model.pickle'

mystem = Mystem()

def prepare_df(df):
    data = df.copy()
    data = data.dropna(subset=['user_id', 'service_id'])
    data = data.drop_duplicates(subset=['user_id', 'service_id'])
    data = data[['user_id', 'service_id']]
    return data

df = pd.read_parquet(PATH_TO_SECTIONS_DF)
data = prepare_df(df)

c_item = CategoricalDtype(sorted(data['service_id'].unique()), ordered=True)
c_user = CategoricalDtype(sorted(data['user_id'].unique()), ordered=True)

data['service_id'] = data['service_id'].astype(c_item)
data['user_id'] = data['user_id'].astype(c_user)

# to categorical
data['user_id'] = data['user_id'].astype(c_user)
data['service_id'] = data['service_id'].astype(c_item)

train_data = csr_matrix(
    (np.ones(len(data)), (data['user_id'].cat.codes, data['service_id'].cat.codes)),
    shape=(len(c_user.categories), len(c_item.categories))
)

with open(PATH_TO_SECTIONS_MODEL, 'rb') as f:
    model = pickle.load(f)


def recommend_by_userid(name, df, data, c_item, c_user, model):
    """
    name это айди юзера для которого хотим рекомендовать
    например 10 или 93126
    """
    user_id = list(c_user.categories).index(name)

    recommended_items = model.recommend(
        user_id,
        train_data,
        N=10,
        filter_already_liked_items=True,
    )

    recommended_items = [list(c_item.categories)[x[0]] for x in recommended_items]
    already = data[data['user_id'] == name]['service_id'].unique().tolist()

    recommended = [df[df['service_id'] == x]['service_name'].unique().tolist()[0] for x in recommended_items]
    already = [df[df['service_id'] == x]['service_name'].unique().tolist()[0] for x in already]
    recomendations = remove_dublicates_by_tokens(already, recommended)

    return already, recomendations


def clean_text(text):
    try:
        if text:
            tokens = mystem.lemmatize(text.lower())
        else:
            return text
    except BrokenPipeError:
        mystem.close()
        mystem.start()
        tokens = mystem.lemmatize(text.lower())

    return [x for x in tokens if x not in ['\n', ' ']]


def remove_dublicates_by_tokens(already, recomendations):
    cleaned_already = [clean_text(x) for x in already]
    cleaned_recomendations = [clean_text(x) for x in recomendations]
    idxs = [idx for idx, x in enumerate(cleaned_recomendations) if x not in cleaned_already]
    return [recomendations[idx] for idx in idxs]


def prepare_recomendations_for_print(already, recomendations):
    text = ('Ранее вы посещали секции:\n'
            + '{} \n\n'.format(already)
            + 'Рекомендуем вам ещё и другие секции:'
            + '\n{}'.format(recomendations))
    return text

def go(name):
    """
    name это айди юзера для которого хотим рекомендовать
    например 10 или 93126
    """
    already, recomendations = recommend_by_userid(name, df, data, c_item, c_user, model)
    #text_answer_for_print = prepare_recomendations_for_print(already, recomendations)
    return already, recomendations


In [12]:
go(10)

'Ранее вы посещали секции:\n[\'Хоровое пение\', \'Хоровое пение\'] \n\nРекомендуем вам ещё и другие секции:\n[\'Фортепиано \', \'уровень I\', \'Музыкальное искусство / Фортепиано / Фортепиано / \', \'Гусли\', \'Академический вокал (5 лет), уровень I\', \'"Английский для всех" микрогруппы 4-5 чел.\', \'4 часа в неделю\', \'Комплексная образовательная программа\', \'6 часов в неделю\']'

Сделаю Flask аппу на Heroku.